# k-nearest neighbour

An implementation of the k-nearest neighbour algorithm.

Sebastian Thomas

In [ ]:
# python
import bisect
from collections import Counter

# data
import numpy as np

# machine learning
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin

In [ ]:
# k-nearest neighbour algorithm
class KNNEstimator(BaseEstimator):
 
    def __init__(self, n_neighbours=5):
        self.n_neighbours = n_neighbours

    def _distance(self, x, y):
        return np.sqrt(np.sum((x - y)**2))
        
    def _neighbour_indices(self, instance):
        distances = []
        
        for (idx, train_instance) in enumerate(self.X_train_):
            # insert dist in the sorted list distances
            bisect.insort(distances, (self._distance(instance, train_instance), idx))
            # reduce length
            if len(distances) > self.n_neighbours:
                distances.pop()
            
        return [idx for (dist, idx) in distances]
    
    def _neighbour_targets(self, instance):
        return [self.y_train_[idx] for idx in self._neighbour_indices(instance)]

    def fit(self, X_train, y_train):
        self.X_train_ = X_train
        self.y_train_ = y_train

In [ ]:
# k-nearest neighbour classifier
class KNNClassifier(KNNEstimator, ClassifierMixin):
    
    def predict(self, X):
        predictions = [Counter(self._neighbour_targets(instance)).most_common(1)[0][0] for instance in X]        
        return np.array(predictions)

In [ ]:
# example
from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

(X, y) = load_iris(return_X_y=True)

(X_train, X_test, y_train, y_test) = train_test_split(X, y, random_state=0)

classifier = make_pipeline(StandardScaler(), KNNClassifier())
classifier.fit(X_train, y_train)

print('accuracy for k = 5:', accuracy_score(y_test, classifier.predict(X_test)))

In [ ]:
# k-nearest neighbour regressor
class KNNRegressor(KNNEstimator, RegressorMixin):
    
    def predict(self, X):
        predictions = [np.array(self._neighbour_targets(instance)).mean() for instance in X]        
        return np.array(predictions)

In [ ]:
# example
import math

from sklearn.datasets import load_boston
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

(X, y) = load_boston(return_X_y=True)

(X_train, X_test, y_train, y_test) = train_test_split(X, y, random_state=0)

regressor = make_pipeline(StandardScaler(), KNNRegressor())
regressor.fit(X_train, y_train)

print('R squared for k=5:', r2_score(y_test, regressor.predict(X_test)))
print('root mean squared error for k=5:', math.sqrt(mean_squared_error(y_test, regressor.predict(X_test))))